# Text Summarizer Example

This is a notebook showing a simple example of text summarization. The model is [DistilBart](https://huggingface.co/sshleifer/distilbart-cnn-6-6) which is a small implementation of [Bart](https://arxiv.org/abs/1910.13461).

In [1]:
!pip install transformers

     |████████████████████████████████| 890kB 5.3MB/s 
     |████████████████████████████████| 1.1MB 27.6MB/s 
     |████████████████████████████████| 3.0MB 39.6MB/s 
     |████████████████████████████████| 890kB 38.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=adbfeadc5a3fb0371b4feb71b7d3ad64d09b2cfb03599ce4addba6e7431445b6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


We're going to use an example news article about an owl that got stuck down a well.

https://www.bbc.com/news/world-europe-53554755

In [26]:
!git clone https://github.com/AMontgomerie/text_summarizer
%cd text_summarizer/colab

Cloning into 'text_summarizer'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.
/content/text_summarizer/colab/text_summarizer/colab/text_summarizer/colab/text_summarizer/colab/text_summarizer/colab


In [27]:
with open('owl_rescue.txt') as file:
    text = file.read()
print(text)

A rescuer in northern Germany brought a trapped owl out of a 40m-deep (131ft) well at a ruined castle, after descending on ropes.

A local had heard the distressed eagle owl hooting from the well on Saturday and alerted the police.

The Bad Segeberg fire service pumped oxygen into the shaft and set up an abseiling rig, after failing to lure the owl into a sack with bait.

The young bird is now safely in the hands of a local bat sanctuary.

Bad Segeberg is a town just north of Lübeck and is famous for the Kalkberg, a massive gypsum rock which is topped by the ruined medieval castle.

The eagle owl rescue involved a team of 12 firefighters, plus a six-member volunteer technical team and two staff from the nearby bat centre, the fire service reported (in German).

They got to the bird just in time, as a probe lowered into the well indicated that there was little air inside. They used a powerful light and telescope to locate the owl, before sending a rescuer down with breathing apparatus.


Now we can initialize the tokenizer and the model. The inputs will be truncated at 512 tokens, so it can only process short articles. The outputs are generated using a beam search with 4 beams and the outputs are limited to 64 tokens.

In [21]:
from transformers import BartTokenizer, BartForConditionalGeneration

MODEL = 'sshleifer/distilbart-cnn-6-6'
INPUT_MAX_LEN = 512
OUTPUT_MAX_LEN = 64
BEAMS = 4

class TextSummarizer():

    def __init__(self):       
        self.model = BartForConditionalGeneration.from_pretrained(MODEL)
        self.tokenizer = BartTokenizer.from_pretrained(MODEL)

    def __call__(self, text):
        inputs = self.tokenizer.encode_plus(
            text,
            max_length=INPUT_MAX_LEN,
            truncation=True,
            return_tensors='pt'
        )

        summary_ids = self.model.generate(
            inputs['input_ids'],
            num_beams=BEAMS,
            max_length=OUTPUT_MAX_LEN,
            early_stopping=True)

        summary = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        return summary


summarizer = TextSummarizer()

In [22]:
summarizer(text)

'A local had heard the distressed eagle owl hooting from the well on Saturday and alerted the police. The Bad Segeberg fire service pumped oxygen into the shaft and set up an abseiling rig. The young bird is now safely in the hands of a local bat sanctuary.'